In [17]:
from PyPDF2 import PdfReader
import glob

for i in range(0,len(glob.glob("*.pdf"))):

    reader = PdfReader(glob.glob("*.pdf")[i])
    page = reader.pages[0]
    text = page.extract_text()
    print(text)

Dynamic dimension reduction
for ﬁnancial applications
D I S S E R T A T I O N
zur Erlangung des akademischen Grades
doctor rerum politicarum
(Doktor der Wirtschaftswissenschaft)
eingereicht an der
Wirtschaftswissenschaftlichen Fakultät
Humboldt-Universität zu Berlin
von
MSc. Sergey  Nasekin
Präsidentin der Humboldt-Universität zu Berlin:
Prof. Dr.-Ing. Dr. Sabine Kunst
Dekan der Wirtschaftswissenschaftlichen Fakultät:
Prof. Dr. Christian D. Schade
Gutachter:
1. Prof. Dr. Wolfgang Karl Härdle, Humboldt-Universität zu Berlin
2. Prof. Yingxing Li, Ph.D., Xiamen University
Tag des Kolloquiums:
Tail Event driven Financial Risk Modelling
D I S S E R T A T I O N
zur Erlangung des akademischen Grades
doctor rerum politicarum
(Doktor der Wirtschaftswissenschaft)
im Fach Statistik
eingereicht an der
Wirtschaftswissenschaftlichen Fakultät
der Humboldt-Universität zu Berlin
von
M.Sc. Lining Yu
Präsidentin der Humboldt-Universität zu Berlin:
Prof. Dr.-Ing. Dr. Sabine Kunst
Dekan der Wirtschaftswiss

Multiple definitions in dictionary at byte 0x75c6fd for key /Lang


Digital and Surrogate Finance
D I S S E R T A T I O N
zur Erlangung des akademischen Grades
doctor rerum politicarum
(Doktor der Wirtschaftswissenschaft)
eingereicht an der
Wirtschaftswissenschaftlichen Fakultät
der Humboldt-Universität zu Berlin
von
Raphael Constantin Georg Reule (Dipl.-Jur.)
geboren am 16. Januar 1989 in München, Bundesrepublik Deutschland
Präsidentin der Humboldt-Universität zu Berlin:
Prof. Dr.-Ing. Dr. Sabine Kunst
Dekan der Wirtschaftswissenschaftlichen Fakultät:
Prof. Dr. Daniel Klapper
Gutachter/Gutachterin: 1. Prof. Dr. Wolfgang Karl Härdle
2. Prof. Dr. Dr. Stefan Grundmann, LL.M.
Tag des Kolloquiums:
E S S AY S O N
U S I NG M AC H I N E L E A R N I NG
F OR C AU S A L I N F E R E N C E
D I S S E RTAT I O N
zur Erlangung des akademischen Grades
doctor rerum politicarum
(Doktor der Wirtschaftswissenschaft)
eingereicht an der
Wirtschaftswissenschaftlichen Fakultät
der Humboldt-Universität zu Berlin
von
Daniel Jacob
geboren am 05.11.1985 in Eutin, Deutschland
Präs